In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
data_path = '/content/drive/MyDrive/ML/COOKIE/info/data/act_dataset/'

df = pd.read_csv(data_path + 'targets_ds.csv', index_col=0)

In [ ]:
import numpy as np
import os

os.chdir(data_path + 'данные_вибрации/2024_06_01/compilled_data/')

⬇ минут 5 выполняется, надо подождать ⬇

In [ ]:
file_names = [f for f in os.listdir() if f.endswith(".npy")]

# Извлекаем времена и данные
vibro_time_list = [f[:-4] for f in file_names]
vibro_data = [np.fromfile(f, dtype=np.float64) for f in file_names]

# Преобразуем время в формате datetime
vibro_time_list = pd.to_datetime(vibro_time_list, format="%Y-%m-%d %H-%M-%S")

In [ ]:
vibro_times_df = pd.DataFrame(vibro_time_list, columns=['t'])
vibro_times_df = vibro_times_df.sort_values(by='t')

vibro_df = vibro_times_df
vibro_df['data'] = pd.Series(vibro_data)

In [ ]:
time_map = vibro_df.set_index('t')['data'].to_dict()

df['data'] = df['t'].map(time_map)

In [ ]:
df.drop('t', axis=1, inplace=True)
df = df.dropna()
df

,gas_flowrate_m3day,liquid_flowrate_m3day,data
0,18.174109,124.918130,"[-257.4310012180081, -238.4309937175642, -220...."
1,24.173818,83.176202,"[-38.32092524887501, -35.32090154746525, -32.3..."
2,36.278781,252.442349,"[158.84741085323552, 166.8474135292099, 170.84..."
3,18.134775,125.538447,"[51.76979772826482, 59.769791075855764, 68.769..."
4,24.163295,167.007924,"[90.0722388229633, 95.07224345916049, 97.07224..."
...,...,...,...
1036,6.588148,86.502914,"[0.09052121553249481, 3.0905346481672726, 9.09..."
1037,6.624843,88.078710,"[-93.28179545594371, -99.28178833994349, -101...."
1038,19.865945,44.411194,"[-128.33106808943387, -136.33110346987198, -14..."
1039,13.211058,88.616245,"[-125.05873544542621, -129.05878400841095, -12..."


In [ ]:
from sklearn.model_selection import train_test_split

for i in range(1,961):
  X = np.fromiter((i for i in df['data'][0]), dtype=np.float64)
X = np.array(np.array_split(960, X, 4), dtype=np.float64).T
X

In [44]:
X.shape

(128000, 4)

In [45]:
X_reshaped = X.reshape(960, 128000, 4)
X_reshaped.shape

ValueError: cannot reshape array of size 512000 into shape (960,128000,4)

In [41]:
y_1.shape

(960, 1)

In [40]:
y_1 = df['liquid_flowrate_m3day'].values
y_2 = df['gas_flowrate_m3day'].values

y_1 = y_1.reshape(-1, 1)
y_2 = y_2.reshape(-1, 1)
y = np.concatenate([y_1, y_2], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


ValueError: Found input variables with inconsistent numbers of samples: [128000, 960]

#FTT Модель

In [ ]:
def conv_block(tensor, filters, kernel_size, activation, strides=1, dropout=0.1):
  conv_layer = tf.keras.layers.Conv1D(filters=filters,
                                      kernel_size=kernel_size,
                                      strides=strides,
                                      padding='same',
                                      activation=None)(tensor)

  batch_norm_layer = tf.keras.layers.BatchNormalization()(conv_layer)
  activation_layer = tf.keras.layers.Activation(activation)(batch_norm_layer)
  dropout_layer = tf.keras.layers.Dropout(dropout)(activation_layer)

  pooling_layer = tf.keras.layers.MaxPool1D(pool_size=2,
                                            strides=2)(dropout_layer)

  return pooling_layer

def dense_block(tensor, units, activation, dropout=0.1):
  dense_layer = tf.keras.layers.Dense(units=units,
                                      activation=None)(tensor)
  batch_norm_layer = tf.keras.layers.BatchNormalization()(dense_layer)
  activation_layer = tf.keras.layers.Activation(activation)(batch_norm_layer)
  dropout_layer = tf.keras.layers.Dropout(dropout)(activation_layer)
  return dropout_layer

def get_model(input_shape=(128000, 4), depth=8):
  input_layer = tf.keras.layers.Input(shape=input_shape)

  #Encoder
  for i in range(depth):
    if i == 0:
      conv = conv_block(tensor=input_layer,
                        filters=32,
                        kernel_size=3,
                        activation='relu',
                        strides=1,
                        dropout=0.1)
    else:
      conv = conv_block(tensor=conv,
                        filters=32,
                        kernel_size=3,
                        activation='relu',
                        strides=1,
                        dropout=0.1)

  #Fully-connected
  flatten_layer = tf.keras.layers.Flatten()(conv)

  dense = dense_block(tensor=flatten_layer,
                      units=100,
                      activation='relu')

  dense = dense_block(tensor=flatten_layer,
                      units=50,
                      activation='relu')

  output_layer = tf.keras.layers.Dense(units=2,
                                      activation='relu')(dense)

  model = tf.keras.models.Model(input_layer, output_layer)

  model.compile(optimizer='adam', loss='mse', metrics=['mape'])

  return model

In [ ]:
ftt_model = get_model()

In [25]:
X_train

,data
310,"[-182.81720943623, -175.81717548945502, -168.8..."
544,"[-19.18153144409028, -20.18153261804661, -19.1..."
302,"[192.27338886170813, 196.27335576931404, 190.2..."
626,"[-59.945069553533756, -56.94507024166135, -54...."
542,"[-86.90234427331134, -92.90234070186413, -95.9..."
...,...
107,"[-59.68620837679002, -57.686188840544595, -55...."
278,"[-36.30352953541379, -32.303531314995496, -29...."
941,"[86.60980755655717, 81.60976776884799, 77.6097..."
512,"[-132.219837456348, -124.2198337719693, -113.2..."


In [21]:
reshaped_X = X_train.to_numpy().reshape(-1, 128000, 4)

ValueError: cannot reshape array of size 768 into shape (128000,4)

In [19]:
ftt_model.fit(tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train), epochs=10, batch_size=32)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
print('Train loss:', history.history['loss'])
print('Train loss:', history.history['mape'])

In [ ]:
loss, mae, mape_value = ftt_model.evaluate(X_test, {'y_1_output': y_test_1, 'y_2_output': y_test_2})

print(f'Test Loss: {loss}, MAE: {mae}, MAPE: {mape_value}')